# Ejemplo de uso de Scalene en Jupyter Notebook

Este notebook muestra cómo usar Scalene para profilear código Python.


In [ ]:
# Paso 1: Cargar la extensión de Scalene
%load_ext scalene


In [ ]:
# Paso 2: Definir una función que queremos profilear
import time
import requests
from pathlib import Path

def download_image(url: str, img_num: int) -> Path:
    """Descarga una imagen y la guarda"""
    print(f"Downloading {url}...")
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    
    filename = f"image_{img_num}.jpg"
    download_path = Path("original_images") / filename
    download_path.parent.mkdir(parents=True, exist_ok=True)
    
    with download_path.open("wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    
    return download_path


In [ ]:
# Paso 3: Profilear una línea específica con %scrun
url = "https://images.unsplash.com/photo-1516117172878-fd2c41f4a759?w=1920&h=1080&fit=crop"
%scrun download_image(url, 1)


In [ ]:
# Paso 4: Profilear una celda completa con %%scalene
%%scalene
from PIL import Image

def process_image(image_path: Path) -> Path:
    """Procesa una imagen (CPU-intensivo)"""
    save_path = Path("processed_images") / image_path.name
    save_path.parent.mkdir(parents=True, exist_ok=True)
    
    with Image.open(image_path) as img:
        data = list(img.getdata())
        width, height = img.size
        new_data = []
        
        # Loop CPU-intensivo que Scalene mostrará
        for i in range(len(data)):
            current_r, current_g, current_b = data[i]
            # Procesamiento simple
            gray = (current_r + current_g + current_b) // 3
            new_data.append((gray, gray, gray))
        
        edge_img = Image.new("RGB", (width, height))
        edge_img.putdata(new_data)
        edge_img.save(save_path)
    
    return save_path

# Ejecutar
if Path("original_images/image_1.jpg").exists():
    result = process_image(Path("original_images/image_1.jpg"))
    print(f"Processed: {result}")


In [ ]:
# Paso 5: Profilear solo CPU (sin memoria)
%%scalene --cpu-only
import time

def cpu_intensive_task():
    total = 0
    for i in range(1000000):
        total += i * i
    return total

result = cpu_intensive_task()
print(f"Result: {result}")


## Notas importantes

- En **macOS**, Scalene no puede profilear procesos secundarios (multiprocessing)
- El profiling de memoria puede estar limitado en notebooks
- Para análisis completo, convierte el notebook a script y usa `scalene script.py`
- Los resultados se muestran directamente en el output de la celda
